In [145]:
%reset -f

In [146]:
from IPython.display import display, HTML
from sklearn import metrics
from sklearn import cross_validation
from scipy import interp
import pandas as pd
import numpy as np
import os

import sys
sys.path.insert(1,"../../src/")
from TypeFeatImputer import TypeFeatImputer
from i_score_parallel import i_score
from TypeFeatFS import DiscreteFS, ContinuousFS
from TypeFeatFilter import DiscreteFilter, ContinuousFilter
from OutlierFiltering import OutlierFiltering
from typeFeat_score import typeFeat_score

from pylab import *
import seaborn as sns
import matplotlib.pyplot as plt


np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:,.2f}'.format
plt.style.use('classic')

%matplotlib inline

In [147]:
ddf = []
pathDir = os.path.join('resources','results') 
path, dirs, files = os.walk(pathDir).next()
file_count = len(files)

for f in files:
    ddf.append(pd.read_pickle(os.path.join('resources','results', str(f) )))
    
dfAll = pd.concat(ddf)
print dfAll.shape

(174, 26)


In [148]:
dfAll.cv_f1_mean = pd.to_numeric(dfAll.cv_f1_mean)
dfAll.cv_rec_mean = pd.to_numeric(dfAll.cv_rec_mean)
dfAll.cv_prec_mean = pd.to_numeric(dfAll.cv_prec_mean)

dfAll.cv_f1_std = pd.to_numeric(dfAll.cv_f1_std)
dfAll.cv_prec_std = pd.to_numeric(dfAll.cv_prec_std)
dfAll.cv_rec_std = pd.to_numeric(dfAll.cv_rec_std)
dfAll.exp = pd.to_numeric(dfAll.exp)

dfAll["cv_f1"] = dfAll["cv_f1_mean"].round(2).astype(str).str.cat(dfAll["cv_f1_std"].round(2).astype(str), sep="+/-")
dfAll["cv_prec"] = dfAll["cv_prec_mean"].round(2).astype(str).str.cat(dfAll["cv_prec_std"].round(2).astype(str), sep="+/-")
dfAll["cv_rec"] = dfAll["cv_rec_mean"].round(2).astype(str).str.cat(dfAll["cv_rec_std"].round(2).astype(str), sep="+/-")
dfAll["num_nones"] = np.sum(np.hstack(((dfAll.sm == "none").reshape(-1,1),(dfAll.fs == "none").reshape(-1,1))), axis=1)

print dfAll.columns.tolist()
print dfAll.shape

['exp', 'typeDisease', 'typeEncounter', 'typeHypothesis', 'typeDataFeatures', 'size_tr', 'fs', 'sm', 'cls', 'metric', 'params', 'tr_f1', 'tr_prec', 'tr_rec', 'cv_f1_mean', 'cv_f1_std', 'cv_prec_mean', 'cv_prec_std', 'cv_rec_mean', 'cv_rec_std', 'test_f1', 'test_prec', 'test_rec', 'test_auc', 'time', 'pipeline', 'cv_f1', 'cv_prec', 'cv_rec', 'num_nones']
(174, 30)


/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/ipykernel_launcher.py:13: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  del sys.path[0]


In [149]:
dfAll.sort_values(["typeDisease","cv_f1_mean"], ascending=False,inplace=True)
dfAll[['typeDisease',"size_tr","sm","metric","cls","params",
     'tr_f1', 'tr_prec', 'tr_rec', 
     'cv_f1_mean', 'cv_prec_mean', 'cv_rec_mean', 
     'test_f1', 'test_prec', 'test_rec', 'test_auc']]

,typeDisease,size_tr,sm,metric,cls,params,tr_f1,tr_prec,tr_rec,cv_f1_mean,cv_prec_mean,cv_rec_mean,test_f1,test_prec,test_rec,test_auc
0,subset,0.20,none,recall,rf,"[gini, 6, balanced, 150]",0.65,0.65,0.65,0.62,0.62,0.62,0.61,0.61,0.61,0.60
0,subset,0.40,none,recall,rf,"[entropy, 6, balanced, 100]",0.63,0.63,0.63,0.62,0.62,0.62,0.61,0.61,0.61,0.60
0,subset,0.20,none,roc_auc,logReg,"[balanced, 0.1, l1]",0.61,0.62,0.61,0.61,0.62,0.61,0.60,0.60,0.61,0.59
0,subset,0.60,none,recall,rf,"[entropy, 6, balanced, 100]",0.63,0.63,0.63,0.61,0.61,0.61,0.62,0.62,0.61,0.60
0,subset,0.40,none,roc_auc,logReg,"[balanced, 0.1, l1]",0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.60,0.61,0.59
0,subset,0.60,none,roc_auc,logReg,"[balanced, 0.01, l2]",0.61,0.61,0.61,0.61,0.61,0.60,0.60,0.60,0.61,0.59
0,subset,0.60,none,recall,logReg,"[balanced, 1e-05, l2]",0.59,0.60,0.58,0.59,0.60,0.58,0.59,0.58,0.60,0.58
0,subset,0.60,none,roc_auc,nb,[],0.58,0.61,0.63,0.59,0.61,0.63,0.58,0.62,0.61,0.56
0,subset,0.60,none,recall,nb,[],0.58,0.61,0.63,0.59,0.61,0.63,0.58,0.62,0.61,0.56
0,subset,0.20,none,recall,logReg,"[balanced, 1e-05, l2]",0.59,0.60,0.59,0.58,0.60,0.58,0.58,0.58,0.60,0.58


In [150]:
dfAll.sort_values(["typeDisease","cv_f1_mean"], ascending=False,inplace=True)
dfAll.groupby(["typeDisease"])[["size_tr","sm","metric","cls","params",
     'tr_f1', 'tr_prec', 'tr_rec', 
     'cv_f1_mean', 'cv_prec_mean', 'cv_rec_mean', 
     'test_f1', 'test_prec', 'test_rec', 'test_auc']].first()

,size_tr,sm,metric,cls,params,tr_f1,tr_prec,tr_rec,cv_f1_mean,cv_prec_mean,cv_rec_mean,test_f1,test_prec,test_rec,test_auc
typeDisease,,,,,,,,,,,,,,,
Circulatory,0.40,none,recall,rf,"[entropy, 6, balanced, 100]",0.63,0.64,0.63,0.61,0.62,0.61,0.60,0.60,0.60,0.59
Diabetis,0.60,none,roc_auc,rf,"[gini, 6, balanced, 500]",0.64,0.65,0.64,0.62,0.63,0.62,0.61,0.61,0.62,0.60
Digestive,0.40,none,recall,rf,"[gini, 4, balanced, 100]",0.65,0.65,0.65,0.60,0.60,0.60,0.61,0.61,0.61,0.58
Genitourinary,0.60,none,recall,rf,"[gini, 4, balanced, 150]",0.63,0.63,0.63,0.60,0.60,0.60,0.60,0.60,0.60,0.59
Muscoskeletal,0.60,none,roc_auc,nb,[],0.62,0.62,0.64,0.62,0.63,0.65,0.60,0.62,0.60,0.56
Neoplasms,0.40,none,roc_auc,nb,[],0.63,0.63,0.65,0.63,0.63,0.66,0.61,0.64,0.60,0.55
Poisoning,0.40,none,roc_auc,rf,"[gini, 4, balanced, 100]",0.67,0.66,0.67,0.60,0.60,0.60,0.62,0.62,0.61,0.59
Respiratory,0.20,after,roc_auc,rf,"[gini, 4, None, 500, 4]",0.65,0.65,0.66,0.60,0.60,0.60,0.60,0.60,0.60,0.59
subset,0.20,none,recall,rf,"[gini, 6, balanced, 150]",0.65,0.65,0.65,0.62,0.62,0.62,0.61,0.61,0.61,0.60
